In [1]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from collections import Counter
from numpy import savez_compressed
from datetime import date
from numpy import load
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Reshape, Lambda, Input, BatchNormalization, concatenate
from sklearn.metrics import accuracy_score
from sklearn import svm
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import random
from sklearn.metrics import accuracy_score
from sklearn import svm
from tensorflow.python.framework.ops import disable_eager_execution

attr_num = 3
seen_num = 10
unseen_num = 2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

In [ ]:
def load_data_make_split(npz_file, train_percentage):
    """
    Load training data (windows + one-hot labels) from compressed file. Split data into train and test set

    Arguments:
        - npz_file: The path to the *.npz file
        - train_percentage: the percentage of data used for training (and not testing), e.g. 0.8
    Returns:
        A 4-tuple of train and test data with labels: (x_train, y_train, x_test, y_test)
    """
    dict_data = load(npz_file)
    x = dict_data['x']
    y = dict_data['y']
    train_length = int(len(x)*train_percentage)
    x_train = x[:train_length]
    y_train = y[:train_length]
    x_test = x[train_length:]
    y_test = y[train_length:]
    return (x_train, y_train, x_test, y_test)

In [ ]:
print(npz_file_seen)
x_train, y_train, z_train, x_test, y_test, z_test = load_data_make_split("{}.npz".format(npz_file_seen),0.8, attr_num )
real_train_x, real_train_y, real_train_z, real_test_x, real_test_y, real_test_z  =  load_data_make_split("{}.npz".format(npz_file_unseen), 0.8,attr_num )
print(y_train.shape)

/content/gdrive/MyDrive/Colab Notebooks/iot_device_classification/npz_windows/balanced_vae_kmeans_seen
(96000,)


In [ ]:
# reshape the label to be onehot form
labels_ohe= np.empty((y_train.shape[0],seen_num,1), dtype=np.float32)
for i in range(y_train.shape[0]):
  label_ohe = np.zeros((1,seen_num))
  label_ohe[0][int(y_train[i])] = 1
  labels_ohe[i] = label_ohe.T

labels_ohe_test= np.empty((y_test.shape[0], seen_num, 1), dtype=np.float32)
for i in range(y_test.shape[0]):
  label_ohe = np.zeros((1,seen_num))
  label_ohe[0][int(y_test[i])] = 1
  labels_ohe_test[i] = label_ohe.T

y_train = labels_ohe.reshape((y_train.shape[0],-1))
y_test = labels_ohe_test.reshape((y_test.shape[0],-1))

disable_eager_execution()

train_size = x_train.shape[0]
batch_size = 64
test_size = x_test.shape[0]

In [ ]:
m = 64 # bacthsize
n_x = 20 # feature dimension
n_y = attr_num  # attribute vector
n_z = 3  # noise dimension
interNo = n_x  # number of neurons in the middle layer
n_epoch = 10  # number of epoches

# encoder input dimension
input_ic = Input(shape=[n_x], name = 'img_class' )
# attribute vector
# cond  = Input(shape=[n_y] , name='class')
temp_h_q = Dense(interNo, activation='relu')(input_ic)
h_q_zd = Dropout(rate=0.2)(temp_h_q)
h_q = Dense(interNo, activation='relu')(h_q_zd)
# dense layer for mu
temp_h_q_2 = Dense(n_z, activation='linear')(h_q)
mu = Dense(n_z, activation='linear')(temp_h_q_2)
# dense layer for log
# log_sigma = Dense(n_z, activation='linear')(h_q)
log_sigma = Dense(n_z, activation='linear')(temp_h_q_2)

def sample_z(args):
    mu, log_sigma = args
    eps = tf.random.normal(shape=[n_z], mean=0., stddev=1.)
    return mu + tf.exp(log_sigma / 2) * eps

z = Lambda(sample_z)([mu, log_sigma])

decoder_hidden = Dense(32, activation='relu')
decoder_out = Dense(n_x, activation='linear')
h_p = decoder_hidden(z)
reconstr = decoder_out(h_p)
vae = Model(inputs=[input_ic], outputs=[reconstr])

encoder = Model(inputs=[input_ic], outputs=[z])

d_in = Input(shape=[n_z])
d_h = decoder_hidden(d_in)
d_out = decoder_out(d_h)
decoder = Model(d_in, d_out)

def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    recon = tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * tf.keras.backend.sum(tf.exp(log_sigma) + tf.keras.backend.square(mu) - 1. - log_sigma, axis=1)
    #print 'kl : ' + str(kl)
    return recon + kl
encoder.summary()
decoder.summary()
vae.compile(optimizer="adam", loss=vae_loss)


X_train = np.concatenate([x_train], axis=1)
print(X_train.shape)
print ('Fitting VAE Model...')
vae.fit({'img_class' : X_train , 'class' : z_train}, x_train, batch_size=m, epochs=n_epoch)

In [ ]:

def generate_reconstruction_loss(x_train, y_train, z_train, encoder, decoder, flag):

  diff_set = dict()
  sess = tf.compat.v1.Session()

  for i in range(200):
    a = x_train[i]
    # flag = 0 seen flag = 1 unseen
    if flag == 0:
      b_1 = y_train[i]
      for j in range(len(b_1)):
        if b_1[j] == 1:
          b = j
    else:
      b =  y_train[i]
    c = z_train[i]
    # print(a, b, c)
    enc_ip = np.concatenate((a, c))
    # print(enc_ip.shape)
    latent = encoder.predict([[enc_ip],[c] ])
    latent = latent.reshape((1,-1))
    c = c.reshape((1,-1))

    dec_ip = np.concatenate((latent, c) , axis=1)
    pseudoTrainData_test = decoder.predict(dec_ip)
    with sess.as_default():
      diff =tf.keras.backend.mean(tf.keras.backend.square(pseudoTrainData_test- a), axis=1).eval()
    if b not in diff_set.keys():
      diff_set[b] = diff
    else:
      before = diff_set[b]
      after = np.concatenate((before,diff))
      diff_set[b] = after
  for i in diff_set.keys():
    print('mean of {} : {} '.format(i,np.mean(diff_set[i])))

  return diff_set


diff_set_seen = generate_reconstruction_loss(x_train, y_train, z_train, encoder, decoder, flag = 0)
diff_set_unseen = generate_reconstruction_loss(real_train_x, real_train_y, real_train_z, encoder, decoder, flag = 1)

In [ ]:
x_train, y_train, z_train, x_test, y_test, z_test = load_data_make_split("{}.npz".format(npz_file_seen),0.8, attr_num )
final_train_x = np.concatenate((real_train_x,x_test),axis = 0)
print(final_train_x.shape)
final_train_y = np.concatenate((real_train_y,y_test),axis = 0)
print(final_train_y.shape)

In [ ]:
def generate_encoder_data(x_test, real_train_x, z_test, real_train_z,real_train_y,y_test ):
  final_train_x = np.concatenate((real_train_x,x_test),axis = 0)
  final_train_z = np.concatenate((real_train_z,z_test),axis = 0)
  final_train_y = np.concatenate((real_train_y,y_test),axis = 0)
  ec_ip = np.concatenate((final_train_x, final_train_z), axis=1)
  # latent = encoder.predict([[enc_ip],[c] ])
  embedding_Data = encoder.predict([ec_ip,final_train_z])
  return embedding_Data, final_train_y

In [ ]:
kmean_z_train, kmean_y_train = generate_encoder_data(x_test, real_train_x, z_test, real_train_z,real_train_y,y_test)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

In [ ]:
Kmean = KMeans(n_clusters=12)
Kmean.fit(kmean_z_train)

In [ ]:
K_label_list = Kmean.labels_
cluster_dict = dict()
for i in range(len(K_label_list)):
  cluster_dict[K_label_list[i]] = []

In [ ]:
for i in range(len(K_label_list)):
  l = cluster_dict[K_label_list[i]]
  l.append(kmean_y_train[i])
  cluster_dict[K_label_list[i]] = l

In [ ]:
total_correct = 0
total_data = 0
total_unseen_correct = 0
total_unseen_data = 0
for i in range(12):
  data_pair =dict()
  unique, count = np.unique(cluster_dict[i], return_counts = True)
  for j in range(len(unique)):
    data_pair[unique[j]] = count[j]
    # print(unique[j], count[j])
  max_count = np.max(count)
  rate = max_count / len(cluster_dict[i])
  total_correct += max_count
  total_data += len(cluster_dict[i])
  max_idx = 0
  for keys in data_pair:
    if data_pair[keys] == max_count:
      max_idx = keys
  if max_idx == 10 or max_idx == 11:
    total_unseen_correct += max_count
    total_unseen_data += len(cluster_dict[i])

  print('cluster: {}, label: {}, accuracy: {} '.format(i, max_idx, rate))
  print(data_pair)

final_acc = total_correct/total_data
final_zsl_acc = total_unseen_correct/total_unseen_data
print('gzsl acc : ', final_acc)
print('zsl acc : ', final_zsl_acc)